# Local version for experiment

이 노트북은 텐서플로우를 이용한 얼굴 인식 모델을 개발하기 위해서 로컬 개발환경에서 모델을 개발하고 테스트하기 위한 노트북이다.
그래서, 학습용 파일들은 로컬 디렉토리에 있는 것을 사용하게 된다. 

In [1]:
import tensorflow as tf
import numpy as np
import re
from datetime import datetime

TRAINING_FILE = '/Users/terrycho/training_data_class5_40/training_file.txt'
VALIDATION_FILE = '/Users/terrycho/training_data_class5_40/validate_file.txt'

flags = tf.app.flags
FLAGS = flags.FLAGS
FLAGS.image_size = 96
FLAGS.image_color = 3
FLAGS.maxpool_filter_size = 2
FLAGS.num_classes=5
FLAGS.batch_size=100
FLAGS.learning_rate = 0.0001
FLAGS.log_dir='/Users/terrycho/'

def get_input_queue(csv_file_name,num_epochs = None):
    train_images = []
    train_labels = []
    for line in open(csv_file_name,'r'):
        cols = re.split(',|\n',line)
        train_images.append(cols[0])
        # 3rd column is label and needs to be converted to int type
        train_labels.append(int(cols[2]) )
                            
    input_queue = tf.train.slice_input_producer([train_images,train_labels],
                                               num_epochs = num_epochs,shuffle = True)
    
    return input_queue

def read_data(input_queue):
    image_file = input_queue[0]
    label = input_queue[1]
    
    image =  tf.image.decode_jpeg(tf.read_file(image_file),channels=FLAGS.image_color)
    
    return image,label,image_file

def read_data_batch(csv_file_name,batch_size=FLAGS.batch_size):
    input_queue = get_input_queue(csv_file_name)
    image,label,file_name= read_data(input_queue)
    image = tf.reshape(image,[FLAGS.image_size,FLAGS.image_size,FLAGS.image_color])
    
    # random image
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image,max_delta=0.5)
    image = tf.image.random_contrast(image,lower=0.2,upper=2.0)
    image = tf.image.random_hue(image,max_delta=0.08)
    image = tf.image.random_saturation(image,lower=0.2,upper=2.0)
    
    batch_image,batch_label,batch_file = tf.train.batch([image,label,file_name],batch_size=batch_size)
    #,enqueue_many=True)
    batch_file = tf.reshape(batch_file,[batch_size,1])

    batch_label_on_hot=tf.one_hot(tf.to_int64(batch_label),
        FLAGS.num_classes, on_value=1.0, off_value=0.0)
    return batch_image,batch_label_on_hot,batch_file

# convolutional network layer 1
def conv1(input_data):
    # layer 1 (convolutional layer)
    FLAGS.conv1_filter_size = 3
    FLAGS.conv1_layer_size = 16
    FLAGS.stride1 = 1
    
    with tf.name_scope('conv_1'):
        W_conv1 = tf.Variable(tf.truncated_normal(
                        [FLAGS.conv1_filter_size,FLAGS.conv1_filter_size,FLAGS.image_color,FLAGS.conv1_layer_size],
                                              stddev=0.1))
        b1 = tf.Variable(tf.truncated_normal(
                        [FLAGS.conv1_layer_size],stddev=0.1))
        h_conv1 = tf.nn.conv2d(input_data,W_conv1,strides=[1,1,1,1],padding='SAME')
        h_conv1_relu = tf.nn.relu(tf.add(h_conv1,b1))
        h_conv1_maxpool = tf.nn.max_pool(h_conv1_relu
                                        ,ksize=[1,2,2,1]
                                        ,strides=[1,2,2,1],padding='SAME')
        
        
    return h_conv1_maxpool

# convolutional network layer 2
def conv2(input_data):
    FLAGS.conv2_filter_size = 3
    FLAGS.conv2_layer_size = 32
    FLAGS.stride2 = 1
    
    with tf.name_scope('conv_2'):
        W_conv2 = tf.Variable(tf.truncated_normal(
                        [FLAGS.conv2_filter_size,FLAGS.conv2_filter_size,FLAGS.conv1_layer_size,FLAGS.conv2_layer_size],
                                              stddev=0.1))
        b2 = tf.Variable(tf.truncated_normal(
                        [FLAGS.conv2_layer_size],stddev=0.1))
        h_conv2 = tf.nn.conv2d(input_data,W_conv2,strides=[1,1,1,1],padding='SAME')
        h_conv2_relu = tf.nn.relu(tf.add(h_conv2,b2))
        h_conv2_maxpool = tf.nn.max_pool(h_conv2_relu
                                        ,ksize=[1,2,2,1]
                                        ,strides=[1,2,2,1],padding='SAME')
        
        
    return h_conv2_maxpool

# convolutional network layer 3
def conv3(input_data):
    FLAGS.conv3_filter_size = 3
    FLAGS.conv3_layer_size = 64
    FLAGS.stride3 = 1
    
    print ('## FLAGS.stride1 ',FLAGS.stride1)
    with tf.name_scope('conv_3'):
        W_conv3 = tf.Variable(tf.truncated_normal(
                        [FLAGS.conv3_filter_size,FLAGS.conv3_filter_size,FLAGS.conv2_layer_size,FLAGS.conv3_layer_size],
                                              stddev=0.1))
        b3 = tf.Variable(tf.truncated_normal(
                        [FLAGS.conv3_layer_size],stddev=0.1))
        h_conv3 = tf.nn.conv2d(input_data,W_conv3,strides=[1,1,1,1],padding='SAME')
        h_conv3_relu = tf.nn.relu(tf.add(h_conv3,b3))
        h_conv3_maxpool = tf.nn.max_pool(h_conv3_relu
                                        ,ksize=[1,2,2,1]
                                        ,strides=[1,2,2,1],padding='SAME')
        
        
    return h_conv3_maxpool

# convolutional network layer 3
def conv4(input_data):
    FLAGS.conv4_filter_size = 5
    FLAGS.conv4_layer_size = 128
    FLAGS.stride4 = 1
    
    with tf.name_scope('conv_4'):
        W_conv4 = tf.Variable(tf.truncated_normal(
                        [FLAGS.conv4_filter_size,FLAGS.conv4_filter_size,FLAGS.conv3_layer_size,FLAGS.conv4_layer_size],
                                              stddev=0.1))
        b4 = tf.Variable(tf.truncated_normal(
                        [FLAGS.conv4_layer_size],stddev=0.1))
        h_conv4 = tf.nn.conv2d(input_data,W_conv4,strides=[1,1,1,1],padding='SAME')
        h_conv4_relu = tf.nn.relu(tf.add(h_conv4,b4))
        h_conv4_maxpool = tf.nn.max_pool(h_conv4_relu
                                        ,ksize=[1,2,2,1]
                                        ,strides=[1,2,2,1],padding='SAME')
        
        
    return h_conv4_maxpool

# fully connected layer 1
def fc1(input_data):
    input_layer_size = 6*6*FLAGS.conv4_layer_size
    FLAGS.fc1_layer_size = 512
    
    with tf.name_scope('fc_1'):
        # 앞에서 입력받은 다차원 텐서를 fcc에 넣기 위해서 1차원으로 피는 작업
        input_data_reshape = tf.reshape(input_data, [-1, input_layer_size])
        W_fc1 = tf.Variable(tf.truncated_normal([input_layer_size,FLAGS.fc1_layer_size],stddev=0.1))
        b_fc1 = tf.Variable(tf.truncated_normal(
                        [FLAGS.fc1_layer_size],stddev=0.1))
        h_fc1 = tf.add(tf.matmul(input_data_reshape,W_fc1) , b_fc1) # h_fc1 = input_data*W_fc1 + b_fc1
        h_fc1_relu = tf.nn.relu(h_fc1)
    
    return h_fc1_relu
    
# fully connected layer 2
def fc2(input_data):
    FLAGS.fc2_layer_size = 256
    
    with tf.name_scope('fc_2'):
        W_fc2 = tf.Variable(tf.truncated_normal([FLAGS.fc1_layer_size,FLAGS.fc2_layer_size],stddev=0.1))
        b_fc2 = tf.Variable(tf.truncated_normal(
                        [FLAGS.fc2_layer_size],stddev=0.1))
        h_fc2 = tf.add(tf.matmul(input_data,W_fc2) , b_fc2) # h_fc1 = input_data*W_fc1 + b_fc1
        h_fc2_relu = tf.nn.relu(h_fc2)
    
    return h_fc2_relu

# final layer
def final_out(input_data):

    with tf.name_scope('final_out'):
        W_fo = tf.Variable(tf.truncated_normal([FLAGS.fc2_layer_size,FLAGS.num_classes],stddev=0.1))
        b_fo = tf.Variable(tf.truncated_normal(
                        [FLAGS.num_classes],stddev=0.1))
        h_fo = tf.add(tf.matmul(input_data,W_fo) , b_fo) # h_fc1 = input_data*W_fc1 + b_fc1
        
    # 최종 레이어에 softmax 함수는 적용하지 않았다. 
        
    return h_fo

# build cnn_graph
def build_model(images,keep_prob):
    # define CNN network graph
    # output shape will be (*,48,48,16)
    r_cnn1 = conv1(images) # convolutional layer 1
    print ("shape after cnn1 ",r_cnn1.get_shape())
    
    # output shape will be (*,24,24,32)
    r_cnn2 = conv2(r_cnn1) # convolutional layer 2
    print ("shape after cnn2 :",r_cnn2.get_shape() )
    
    # output shape will be (*,12,12,64)
    r_cnn3 = conv3(r_cnn2) # convolutional layer 3
    print ("shape after cnn3 :",r_cnn3.get_shape() )

    # output shape will be (*,6,6,128)
    r_cnn4 = conv4(r_cnn3) # convolutional layer 4
    print ("shape after cnn4 :",r_cnn4.get_shape() )
    
    # fully connected layer 1
    r_fc1 = fc1(r_cnn4)
    print ("shape after fc1 :",r_fc1.get_shape() )

    # fully connected layer2
    r_fc2 = fc2(r_fc1)
    print ("shape after fc2 :",r_fc2.get_shape() )
    
    ## drop out
    # 참고 http://stackoverflow.com/questions/34597316/why-input-is-scaled-in-tf-nn-dropout-in-tensorflow
    # 트레이닝시에는 keep_prob < 1.0 , Test 시에는 1.0으로 한다. 
    r_dropout = tf.nn.dropout(r_fc2,keep_prob)
    print ("shape after dropout :",r_dropout.get_shape() ) 
    
    # final layer
    r_out = final_out(r_dropout)
    print ("shape after final layer :",r_out.get_shape() )


    return r_out 

def main(argv=None):
    
    # define placeholders for image data & label for traning dataset
    
    images = tf.placeholder(tf.float32,[None,FLAGS.image_size,FLAGS.image_size,FLAGS.image_color])
    labels = tf.placeholder(tf.int32,[None,FLAGS.num_classes])
    image_batch,label_batch,file_batch = read_data_batch(TRAINING_FILE) 

    keep_prob = tf.placeholder(tf.float32) # dropout ratio
    prediction = build_model(images,keep_prob)
    # define loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=labels))

    tf.summary.scalar('loss',loss)

    #define optimizer
    optimizer = tf.train.AdamOptimizer(FLAGS.learning_rate)
    train = optimizer.minimize(loss)


    # for validation
    #with tf.name_scope("prediction"):
 
    validate_image_batch,validate_label_batch,validate_file_batch = read_data_batch(VALIDATION_FILE)
    label_max = tf.argmax(labels,1)
    pre_max = tf.argmax(prediction,1)
    correct_pred = tf.equal(tf.argmax(prediction,1),tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
            
    tf.summary.scalar('accuracy',accuracy)
        
    
    startTime = datetime.now()
    
    #build the summary tensor based on the tF collection of Summaries
    summary = tf.summary.merge_all()
    
    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
        saver = tf.train.Saver() # create saver to store training model into file
        summary_writer = tf.summary.FileWriter(FLAGS.log_dir,sess.graph)
        
        init_op = tf.global_variables_initializer() # use this for tensorflow 0.12rc0
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        sess.run(init_op)
        
        for i in range(10000):
            images_,labels_ = sess.run([image_batch,label_batch])
            #sess.run(train_step,feed_dict={images:images_,labels:labels_,keep_prob:0.8})
            sess.run(train,feed_dict={images:images_,labels:labels_,keep_prob:0.7})
            
            if i % 10 == 0:
                now = datetime.now()-startTime
                print('## time:',now,' steps:',i)         
                
                # print out training status
                rt = sess.run([label_max,pre_max,loss,accuracy],feed_dict={images:images_ 
                                                          , labels:labels_
                                                          , keep_prob:1.0})
                print ('Prediction loss:',rt[2],' accuracy:',rt[3])
                # validation steps
                validate_images_,validate_labels_ = sess.run([validate_image_batch,validate_label_batch])
                rv = sess.run([label_max,pre_max,loss,accuracy],feed_dict={images:validate_images_ 
                                                          , labels:validate_labels_
                                                          , keep_prob:1.0})
                print ('Validation loss:',rv[2],' accuracy:',rv[3])
                if(rv[3] > 0.9):
                    break
                # validation accuracy
                summary_str = sess.run(summary,feed_dict={images:validate_images_ 
                                                          , labels:validate_labels_
                                                          , keep_prob:1.0})
                summary_writer.add_summary(summary_str,i)
                summary_writer.flush()
        
        saver.save(sess, 'face_recog') # save session
        coord.request_stop()
        coord.join(threads)
        print('finish')
    
main()
    

('shape after cnn1 ', TensorShape([Dimension(None), Dimension(48), Dimension(48), Dimension(16)]))
('shape after cnn2 :', TensorShape([Dimension(None), Dimension(24), Dimension(24), Dimension(32)]))
('## FLAGS.stride1 ', 1)
('shape after cnn3 :', TensorShape([Dimension(None), Dimension(12), Dimension(12), Dimension(64)]))
('shape after cnn4 :', TensorShape([Dimension(None), Dimension(6), Dimension(6), Dimension(128)]))
('shape after fc1 :', TensorShape([Dimension(None), Dimension(512)]))
('shape after fc2 :', TensorShape([Dimension(None), Dimension(256)]))
('shape after dropout :', TensorShape([Dimension(None), Dimension(256)]))
('shape after final layer :', TensorShape([Dimension(None), Dimension(5)]))
('## time:', datetime.timedelta(0, 2, 961004), ' steps:', 0)
('Prediction loss:', 608.31244, ' accuracy:', 0.16)
('Validation loss:', 481.90189, ' accuracy:', 0.23)
('## time:', datetime.timedelta(0, 14, 165824), ' steps:', 10)
('Prediction loss:', 205.54933, ' accuracy:', 0.23)
('Valid

('## time:', datetime.timedelta(0, 556, 59438), ' steps:', 430)
('Prediction loss:', 1.5922632, ' accuracy:', 0.34)
('Validation loss:', 2.1175208, ' accuracy:', 0.30000001)
('## time:', datetime.timedelta(0, 571, 769498), ' steps:', 440)
('Prediction loss:', 1.638521, ' accuracy:', 0.38)
('Validation loss:', 1.7174768, ' accuracy:', 0.41)
('## time:', datetime.timedelta(0, 588, 335443), ' steps:', 450)
('Prediction loss:', 1.438706, ' accuracy:', 0.40000001)
('Validation loss:', 1.942384, ' accuracy:', 0.28999999)
('## time:', datetime.timedelta(0, 605, 872782), ' steps:', 460)
('Prediction loss:', 1.590072, ' accuracy:', 0.31999999)
('Validation loss:', 1.8574281, ' accuracy:', 0.34999999)
('## time:', datetime.timedelta(0, 622, 798857), ' steps:', 470)
('Prediction loss:', 1.4363337, ' accuracy:', 0.41999999)
('Validation loss:', 1.9957557, ' accuracy:', 0.31)
('## time:', datetime.timedelta(0, 638, 265942), ' steps:', 480)
('Prediction loss:', 1.5099189, ' accuracy:', 0.34)
('Valid

('Validation loss:', 1.6355077, ' accuracy:', 0.36000001)
('## time:', datetime.timedelta(0, 1233, 320481), ' steps:', 900)
('Prediction loss:', 1.3725201, ' accuracy:', 0.33000001)
('Validation loss:', 1.9135067, ' accuracy:', 0.30000001)
('## time:', datetime.timedelta(0, 1247, 283126), ' steps:', 910)
('Prediction loss:', 1.3840051, ' accuracy:', 0.37)
('Validation loss:', 1.5467596, ' accuracy:', 0.31)
('## time:', datetime.timedelta(0, 1261, 383642), ' steps:', 920)
('Prediction loss:', 1.488695, ' accuracy:', 0.30000001)
('Validation loss:', 1.8920306, ' accuracy:', 0.31)
('## time:', datetime.timedelta(0, 1275, 258172), ' steps:', 930)
('Prediction loss:', 1.4618504, ' accuracy:', 0.28)
('Validation loss:', 1.6802255, ' accuracy:', 0.31)
('## time:', datetime.timedelta(0, 1289, 29759), ' steps:', 940)
('Prediction loss:', 1.3810269, ' accuracy:', 0.36000001)
('Validation loss:', 1.7387624, ' accuracy:', 0.34)
('## time:', datetime.timedelta(0, 1302, 760222), ' steps:', 950)
('Pr

('## time:', datetime.timedelta(0, 1871, 59586), ' steps:', 1360)
('Prediction loss:', 1.2336084, ' accuracy:', 0.44999999)
('Validation loss:', 2.0715644, ' accuracy:', 0.38999999)
('## time:', datetime.timedelta(0, 1885, 28412), ' steps:', 1370)
('Prediction loss:', 1.409021, ' accuracy:', 0.40000001)
('Validation loss:', 1.3431479, ' accuracy:', 0.43000001)
('## time:', datetime.timedelta(0, 1899, 158858), ' steps:', 1380)
('Prediction loss:', 1.3453869, ' accuracy:', 0.47)
('Validation loss:', 1.5436896, ' accuracy:', 0.33000001)
('## time:', datetime.timedelta(0, 1913, 447173), ' steps:', 1390)
('Prediction loss:', 1.4746948, ' accuracy:', 0.33000001)
('Validation loss:', 1.7825388, ' accuracy:', 0.34999999)
('## time:', datetime.timedelta(0, 1927, 604629), ' steps:', 1400)
('Prediction loss:', 1.2680764, ' accuracy:', 0.46000001)
('Validation loss:', 2.6326635, ' accuracy:', 0.34)
('## time:', datetime.timedelta(0, 1942, 147184), ' steps:', 1410)
('Prediction loss:', 1.3071039, '

('Prediction loss:', 1.25752, ' accuracy:', 0.44)
('Validation loss:', 1.582842, ' accuracy:', 0.38999999)
('## time:', datetime.timedelta(0, 2523, 430283), ' steps:', 1830)
('Prediction loss:', 1.1475161, ' accuracy:', 0.57999998)
('Validation loss:', 1.6489136, ' accuracy:', 0.41999999)
('## time:', datetime.timedelta(0, 2537, 186789), ' steps:', 1840)
('Prediction loss:', 1.2123216, ' accuracy:', 0.55000001)
('Validation loss:', 1.7887683, ' accuracy:', 0.38999999)
('## time:', datetime.timedelta(0, 2550, 996134), ' steps:', 1850)
('Prediction loss:', 1.1490997, ' accuracy:', 0.52999997)
('Validation loss:', 2.1279948, ' accuracy:', 0.38)
('## time:', datetime.timedelta(0, 2564, 921729), ' steps:', 1860)
('Prediction loss:', 1.1275059, ' accuracy:', 0.49000001)
('Validation loss:', 1.4174198, ' accuracy:', 0.43000001)
('## time:', datetime.timedelta(0, 2579, 292204), ' steps:', 1870)
('Prediction loss:', 1.2843273, ' accuracy:', 0.41999999)
('Validation loss:', 1.5155995, ' accuracy

...

('## time:', datetime.timedelta(0, 6940, 198076), ' steps:', 5010)
('Prediction loss:', 0.31747594, ' accuracy:', 0.88999999)
('Validation loss:', 1.0266913, ' accuracy:', 0.68000001)
('## time:', datetime.timedelta(0, 6954, 135105), ' steps:', 5020)
('Prediction loss:', 0.25468898, ' accuracy:', 0.89999998)
('Validation loss:', 0.64356565, ' accuracy:', 0.77999997)
('## time:', datetime.timedelta(0, 6968, 187186), ' steps:', 5030)
('Prediction loss:', 0.29725626, ' accuracy:', 0.91000003)
('Validation loss:', 1.1741208, ' accuracy:', 0.75)
('## time:', datetime.timedelta(0, 6982, 17133), ' steps:', 5040)
('Prediction loss:', 0.35176179, ' accuracy:', 0.88999999)
('Validation loss:', 1.4758643, ' accuracy:', 0.67000002)
('## time:', datetime.timedelta(0, 6996, 106646), ' steps:', 5050)
('Prediction loss:', 0.2493913, ' accuracy:', 0.94)
('Validation loss:', 0.90398526, ' accuracy:', 0.72000003)
('## time:', datetime.timedelta(0, 7009, 867258), ' steps:', 5060)
('Prediction loss:', 0.38

('## time:', datetime.timedelta(0, 7586, 180922), ' steps:', 5460)
('Prediction loss:', 0.49606284, ' accuracy:', 0.85000002)
('Validation loss:', 1.0443231, ' accuracy:', 0.76999998)
('## time:', datetime.timedelta(0, 7600, 761462), ' steps:', 5470)
('Prediction loss:', 0.28533736, ' accuracy:', 0.91000003)
('Validation loss:', 0.79422951, ' accuracy:', 0.80000001)
('## time:', datetime.timedelta(0, 7614, 715332), ' steps:', 5480)
('Prediction loss:', 0.26577497, ' accuracy:', 0.92000002)
('Validation loss:', 0.63009143, ' accuracy:', 0.76999998)
('## time:', datetime.timedelta(0, 7628, 651550), ' steps:', 5490)
('Prediction loss:', 0.35950983, ' accuracy:', 0.88999999)
('Validation loss:', 0.75774604, ' accuracy:', 0.76999998)
('## time:', datetime.timedelta(0, 7642, 577590), ' steps:', 5500)
('Prediction loss:', 0.30365103, ' accuracy:', 0.87)
('Validation loss:', 0.60140789, ' accuracy:', 0.75999999)
('## time:', datetime.timedelta(0, 7656, 509132), ' steps:', 5510)
('Prediction lo

('Prediction loss:', 0.15294129, ' accuracy:', 0.98000002)
('Validation loss:', 0.58192039, ' accuracy:', 0.82999998)
('## time:', datetime.timedelta(0, 8229, 68805), ' steps:', 5920)
('Prediction loss:', 0.23323187, ' accuracy:', 0.93000001)
('Validation loss:', 0.57299781, ' accuracy:', 0.79000002)
('## time:', datetime.timedelta(0, 8243, 126030), ' steps:', 5930)
('Prediction loss:', 0.16833557, ' accuracy:', 0.94999999)
('Validation loss:', 1.1100208, ' accuracy:', 0.81)
('## time:', datetime.timedelta(0, 8257, 318004), ' steps:', 5940)
('Prediction loss:', 0.19002552, ' accuracy:', 0.93000001)
('Validation loss:', 0.74882859, ' accuracy:', 0.80000001)
('## time:', datetime.timedelta(0, 8271, 469505), ' steps:', 5950)
('Prediction loss:', 0.2876454, ' accuracy:', 0.92000002)
('Validation loss:', 1.1289822, ' accuracy:', 0.75999999)
('## time:', datetime.timedelta(0, 8285, 452424), ' steps:', 5960)
('Prediction loss:', 0.19063278, ' accuracy:', 0.92000002)
('Validation loss:', 1.014

('Prediction loss:', 0.15203784, ' accuracy:', 0.94)
('Validation loss:', 0.72400469, ' accuracy:', 0.79000002)
('## time:', datetime.timedelta(0, 8851, 793066), ' steps:', 6370)
('Prediction loss:', 0.17047206, ' accuracy:', 0.93000001)
('Validation loss:', 1.2583271, ' accuracy:', 0.77999997)
('## time:', datetime.timedelta(0, 8865, 770574), ' steps:', 6380)
('Prediction loss:', 0.12688673, ' accuracy:', 0.95999998)
('Validation loss:', 0.80238527, ' accuracy:', 0.83999997)
('## time:', datetime.timedelta(0, 8879, 580600), ' steps:', 6390)
('Prediction loss:', 0.13270062, ' accuracy:', 0.97000003)
('Validation loss:', 0.64593416, ' accuracy:', 0.83999997)
('## time:', datetime.timedelta(0, 8893, 362967), ' steps:', 6400)
('Prediction loss:', 0.12510011, ' accuracy:', 0.97000003)
('Validation loss:', 0.67930448, ' accuracy:', 0.85000002)
('## time:', datetime.timedelta(0, 8907, 157235), ' steps:', 6410)
('Prediction loss:', 0.075684898, ' accuracy:', 0.99000001)
('Validation loss:', 1

('Prediction loss:', 0.11803013, ' accuracy:', 0.95999998)
('Validation loss:', 0.95212287, ' accuracy:', 0.82999998)
('## time:', datetime.timedelta(0, 9473, 718655), ' steps:', 6820)
('Prediction loss:', 0.051386572, ' accuracy:', 1.0)
('Validation loss:', 1.0859307, ' accuracy:', 0.80000001)
('## time:', datetime.timedelta(0, 9487, 490741), ' steps:', 6830)
('Prediction loss:', 0.14820409, ' accuracy:', 0.95999998)
('Validation loss:', 0.75478637, ' accuracy:', 0.82999998)
('## time:', datetime.timedelta(0, 9501, 287798), ' steps:', 6840)
('Prediction loss:', 0.076347858, ' accuracy:', 0.98000002)
('Validation loss:', 0.94828433, ' accuracy:', 0.76999998)
('## time:', datetime.timedelta(0, 9514, 995110), ' steps:', 6850)
('Prediction loss:', 0.14047286, ' accuracy:', 0.98000002)
('Validation loss:', 1.0078058, ' accuracy:', 0.77999997)
('## time:', datetime.timedelta(0, 9528, 805913), ' steps:', 6860)
('Prediction loss:', 0.13317505, ' accuracy:', 0.94999999)
('Validation loss:', 1.

Train과 validation summary 나눠서 쓰는 예제
https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/mnist/mnist_with_summaries.py
